In [67]:
from google.cloud import bigquery
from google.cloud import storage
import json

In [68]:
bgclient = bigquery.Client ()
gsclient = storage.Client ()

In [76]:
project = "udemydataengineering"
dataset = "udemy_retail_gcpdataset"
table1 = f"{project}.{dataset}.udemy_retail_gcptableorderitem"
table2 = f"{project}.{dataset}.udemy_retail_gcptableproduct"

In [70]:
#help (bgclient)
#help (gsclient)

In [71]:

bucket = "udemy-retail-gcpbucket"
folder = "udemy-pythondemo/retail-db/retail_db/"


In [72]:
blobs = list (gsclient.list_blobs (bucket))#list (gsclient.get_bucket (bucket).list_blobs ())#

blobschema = {}

uri1 =""#f"gs://{bucket}/{folder}/order_items/part-00000"
uri2 =""#f"gs://{bucket}/{folder}/products/part-00000"


i = 0

while i < len (blobs):
    '''
    print (type (blobs [i]))
    print (blobs [i])
    print (blobs [i].bucket.name)
    print (blobs [i].name)
    #print (json.load (open (f"gs://{bucket}/{blobs [i].name}")))
    '''
    if ".json" in blobs [i].name:
        #print ("JSON File Found")
        #print  (json.loads (blobs [i].download_as_string (client=gsclient)))#have to use json.loads not json.load because have to download as string when reading from gcp storage
        blobschema = json.loads (blobs [i].download_as_string (client=gsclient))
    elif "order_items" in blobs [i].name:
        uri1 = f"gs://{blobs [i].bucket.name}/{blobs [i].name}"
    elif "products" in blobs [i].name:
        uri2 = f"gs://{blobs [i].bucket.name}/{blobs [i].name}"
    
    i = i+1

#help (gsclient)

In [73]:
#print (blobschema)
schema1raw = sorted (blobschema ["order_items"], key=lambda data: data ["column_position"])
#print (schema1raw)
schema2raw = sorted (blobschema ["products"], key=lambda data: data ["column_position"])
#print (schema2raw)

schema1cols = [schema1coldict ["column_name"] for schema1coldict in schema1raw]
#print (schema1cols)
schema2cols = [schema2coldict ["column_name"] for schema2coldict in schema2raw]
#print (schema2cols)

schema1dictionary = {"orderitem_id": "INTEGER", "orderitem_orderid": "INTEGER", "orderitem_productid": "INTEGER", "orderitem_quantity": "INTEGER", "orderitem_subtotal": "DECIMAL", "orderitem_productprice": "DECIMAL"}
schema2dictionary = {"product_id": "INTEGER", "product_categoryid": "INTEGER", "product_name": "STRING", "product_description": "STRING", "product_price": "DECIMAL", "product_image": "STRING"}

schema1 = []
schema2 = []

schema1keys = list (schema1dictionary.keys ())
i = 0
while i < len (schema1keys):
    schema1append = bigquery.SchemaField (schema1keys [i], schema1dictionary [schema1keys [i]])
    schema1.append (schema1append)
    i = i+1

schema2keys = list (schema2dictionary.keys ())
i = 0
while i < len (schema2keys):
    schema2append = bigquery.SchemaField (schema2keys [i], schema2dictionary [schema2keys [i]])
    schema2.append (schema2append)
    i = i + 1

#print (schema1)
#print (schema2)


In [77]:
jobconfig1 = bigquery.LoadJobConfig (schema=schema1)
jobconfig2 = bigquery.LoadJobConfig (schema=schema2)

#job1 = bgclient.create_job ()
#job2 = bgclient.create_job ()

load1init = bgclient.load_table_from_uri (uri1, table1, job_config=jobconfig1)
load2init = bgclient.load_table_from_uri (uri2, table2, job_config=jobconfig2)

load1result = load1init.result ()
load2result = load2init.result ()

In [85]:
#help (bgclient)

query1string = f"select * from {table1} limit 10"
query2string = f"select * from {table2} limit 10"

query1init = bgclient.query (query1string)
query2init = bgclient.query (query2string)

query1result = query1init.result ()
query2result = query2init.result ()

print (query1result)
print (query2result)

query1list = list (query1result)

i = 0
while i < len (query1list):
    print (query1list [i].values ())
    i = i+1

query2list = list (query2result)

i=0
while i < len (query2list):
    print (query2list [i].values ())
    i = i+1

(168758, 67504, 768, 1, Decimal('299.99'), Decimal('299.99'))
(168865, 67543, 768, 1, Decimal('299.99'), Decimal('299.99'))
(168888, 67552, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169154, 67658, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169218, 67682, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169252, 67695, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169290, 67713, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169355, 67740, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169406, 67757, 768, 1, Decimal('299.99'), Decimal('299.99'))
(169606, 67839, 768, 1, Decimal('299.99'), Decimal('299.99'))
(934, 42, 'Callaway X Hot Driver', None, Decimal('0'), 'http://images.acmesports.sports/Callaway+X+Hot+Driver')
(41, 3, "adidas Men's Mexico Home Soccer Jersey", None, Decimal('90'), 'http://images.acmesports.sports/adidas+Men%27s+Mexico+Home+Soccer+Jersey')
(519, 24, "adidas Men's Mexico Home Soccer Jersey", None, Decimal('90'), 'http://images.acmesports.sports/adidas+Men%27s+

In [86]:
help (bgclient)

Help on Client in module google.cloud.bigquery.client object:

class Client(google.cloud.client.ClientWithProject)
 |  Client(project=None, credentials=None, _http=None, location=None, default_query_job_config=None, default_load_job_config=None, client_info=None, client_options=None) -> None
 |  
 |  Client to bundle configuration needed for API requests.
 |  
 |  Args:
 |      project (Optional[str]):
 |          Project ID for the project which the client acts on behalf of.
 |          Will be passed when creating a dataset / job. If not passed,
 |          falls back to the default inferred from the environment.
 |      credentials (Optional[google.auth.credentials.Credentials]):
 |          The OAuth2 Credentials to use for this client. If not passed
 |          (and if no ``_http`` object is passed), falls back to the
 |          default inferred from the environment.
 |      _http (Optional[requests.Session]):
 |          HTTP object to make requests. Can be any object that
 |   

In [87]:
help (bigquery.Table)

Help on class Table in module google.cloud.bigquery.table:

class Table(_TableBase)
 |  Table(table_ref, schema=None) -> None
 |  
 |  Tables represent a set of rows whose values correspond to a schema.
 |  
 |  See
 |  https://cloud.google.com/bigquery/docs/reference/rest/v2/tables#resource-table
 |  
 |  Args:
 |      table_ref (Union[google.cloud.bigquery.table.TableReference, str]):
 |          A pointer to a table. If ``table_ref`` is a string, it must
 |          included a project ID, dataset ID, and table ID, each separated
 |          by ``.``.
 |      schema (Optional[Sequence[Union[                 :class:`~google.cloud.bigquery.schema.SchemaField`,                 Mapping[str, Any]         ]]]):
 |          The table's schema. If any item is a mapping, its content must be
 |          compatible with
 |          :meth:`~google.cloud.bigquery.schema.SchemaField.from_api_repr`.
 |  
 |  Method resolution order:
 |      Table
 |      _TableBase
 |      builtins.object
 |  
 |  

In [93]:
table0 = f"{project}.{dataset}.udemy_retail_gcptableorders"
delete0table = bigquery.Table.from_string (table0)
delete0query = bgclient.delete_table (delete0table, not_found_ok=True)

delete1query = bgclient.query (f"DROP TABLE IF EXISTS {table1}")

delete2query = bgclient.delete_table (table2, not_found_ok=True)

print (delete0query)
#delete0result = delete0query.result ()
delete1result = delete1query.result ()
print (delete1result)
#delete2result = delete2query.result ()

None
